### Object tracking with opencv

In [ ]:
import cv2

In [ ]:
def initialize(videopath):
    cap = cv2.VideoCapture(videopath)
    od = cv2.createBackgroundSubtractorMOG2(varThreshold=1000, history=20, detectShadows=False) 
    return cap, od

#### custom tracking with ROI

In [ ]:
cap, od =initialize('../data/aerial_tennis_video.mp4')

ret, frame_n = cap.read()  # Read the frame
bbox=cv2.selectROI(frame_n)
tracker=cv2.legacy.TrackerMOSSE_create()
# tracker = cv2.legacy.TrackerTLD_create()
ok = tracker.init(frame_n ,bbox)
while True:
    ret, frame = cap.read() 
    if not ret:
        break
    ok, bbox = tracker.update(frame)
    if ok:
        p1 = (int(bbox[0]), int(bbox[1]))
        p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
        cv2.rectangle(frame, p1, p2, (0,0,255), 2, 1)
    else :
        cv2.putText(frame, "Tracking failure detected", (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)
    cv2.putText(frame, "Mosse" + " Tracker", (100,20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50),2)
    cv2.imshow("Tracking", frame)
    key = cv2.waitKey(1)
    if key == ord('q'):                                         # exit
        break        
    elif key == ord('x'):                                       # pauze screen
        cv2.waitKey(0)            

cap.release()
cv2.destroyAllWindows()
[cv2.waitKey(1) for i in range(1,5)]           


#### Old (useless) code

In [ ]:
cap, od =initialize('../data/aerial_tennis_video.mp4')
ret, frame_n = cap.read()  # Read the frame
bbox=cv2.selectROI(frame_n)
tracker=cv2.legacy.TrackerMOSSE_create()
ok = tracker.init(frame_n ,bbox)
while True:
    ret, frame = cap.read() 
    mask = od.apply(frame)                                      # run mask (black and white)
    _, mask = cv2.threshold(mask, 254, 255,cv2.THRESH_BINARY)   # set treshold to only get really contrasting values in mask
    cv2.imshow('mask', mask)    
    if not ret:
        break
    ok, bbox = tracker.update(frame)
    if ok:
        print('tracking!')
        p1 = (int(bbox[0]), int(bbox[1]))
        p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
        cv2.rectangle(frame, p1, p2, (0,0,255), 2, 1)
    else :
        print('not tracked')
        cv2.putText(frame, "Tracking failure detected", (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)
    cv2.putText(frame, "Mosse" + " Tracker", (100,20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50),2)
    cv2.imshow("Tracking", frame)
    key = cv2.waitKey(1)
    if key == ord('q'):                                         # exit
        break        
    elif key == ord('x'):                                       # pauze screen
        cv2.waitKey(0)            

cap.release()
cv2.destroyAllWindows()
[cv2.waitKey(1) for i in range(1,5)]           


In [ ]:
def cleanup_frame(frame, od):
    # frame = cv2.resize(frame, (960, 540))                             # Resize image
    mask = od.apply(frame)                                              # run mask (black and white)
    _, extra_mask = cv2.threshold(mask, 250, 255, cv2.THRESH_BINARY)    # set treshold to only get really contrasting values in mask
    return extra_mask

In [ ]:
def identifying_contours_players(c, od):
    count = 0
    frame_nr = 6
    while True:
        count +=1
        print(c.get(1))     
        ret, frame = c.read()                                       # read frames
        if not ret:
            print("Stream ended. Exiting...")
            break        
        key = cv2.waitKey(1)
        if key == ord('q'):                                         # exit
            break        
        elif key == ord('x'):                                       # pauze screen
            cv2.waitKey(0)        
        # frame = cv2.resize(frame, (960, 540))                       # Resize image
        # cv2.imshow('frame', frame)
        mask = od.apply(frame)                                      # run mask (black and white)
        _, mask = cv2.threshold(mask, 254, 255,cv2.THRESH_BINARY)   # set treshold to only get really contrasting values in mask
        cv2.imshow('mask', mask)
        contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) #Each individual contour is a Numpy array of (x,y) coordinates of boundary points of the object.
        frame_detect = []
        if count == frame_nr:
            for cnt in contours:
                x,y,w,h = cv2.boundingRect(cnt)                         # get boundingbox
                frame_detect.append([x,y,w,h])
                cv2.rectangle(frame,(x,y), (x+w, y+h),(0,0, 255), 3)     # draw rectangle
                cv2.imshow('frame2', frame)  
                cv2.imwrite('test.png', frame)  
            cv2.waitKey(0)      
            print(frame_detect)
            break
            # break
    c.release()
    cv2.destroyAllWindows()
    [cv2.waitKey(1) for i in range(1,5)]                            # needed to destroy window
    return frame_detect

In [ ]:
# cap, od =initialize('../data/aerial_football_video.mp4')
cap, od =initialize('../data/aerial_tennis_video.mp4')
frame_bbox = identifying_contours_players(cap, od)

In [ ]:
len(frame_bbox)

In [ ]:
frame_bbox_large = []
for bb in frame_bbox:
    widthlarge = bb[2] * 50
    heightlarge = bb[3] * 50
    frame_bbox_large.append((bb[0], bb[1], widthlarge, heightlarge))

In [ ]:
frame_bbox_large[:5]

In [ ]:
cap, od =initialize('../data/aerial_tennis_video.mp4')
ret, frame_n = cap.read()  # Read the frame

for start_contour in frame_bbox_large:
    bbox = start_contour
    # bbox = (546, 109, 150, 150)
    tracker=cv2.legacy.TrackerMOSSE_create()
    ok = tracker.init(frame_n ,bbox)
    first = None
    while True:
        if not first:
            cap.set(1,n+1) #otherwise starts  reading from 22 when starting with 20
            first ='done'
        ret, frame = cap.read() 
        # frame = cleanup_frame(frame, od)
        # print(cap.get(1))
        if not ret:
            break
        ok, bbox = tracker.update(frame)
        if ok:
            print('tracking!')
            p1 = (int(bbox[0]), int(bbox[1]))
            p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
            cv2.rectangle(frame, p1, p2, (0,0,255), 2, 1)
        else :
            print('not tracked')
            cv2.putText(frame, "Tracking failure detected", (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)
        cv2.putText(frame, "Mosse" + " Tracker", (100,20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50),2)
        cv2.imshow("Tracking", frame)
        key = cv2.waitKey(1)
        if key == ord('q'):                                         # exit
            break        
        elif key == ord('x'):                                       # pauze screen
            cv2.waitKey(0)            

    # cap.release()
    # cv2.destroyAllWindows()
    # [cv2.waitKey(1) for i in range(1,5)]           


In [ ]:
#(424, 278, 54, 32) frame 20 starting point tracking bbox